In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pickle
with open('/content/drive/My Drive/Flipkart/output.bin', 'rb') as f:
    imgs = pickle.load(f)

In [0]:
path = '/content/drive/My Drive/Flipkart/'

In [0]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,Lambda
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D,UpSampling2D,Concatenate,Conv2DTranspose
from keras.layers import Conv2D, Flatten, Dense, MaxPool2D,MaxPooling2D, Activation, Dropout, BatchNormalization, Input
from keras.models import Model,Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.callbacks import LearningRateScheduler,Callback
from keras.optimizers import RMSprop
import glob
import cv2
import os
from math import *
from keras.applications.vgg16 import VGG16 
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import h5py
from keras.utils import to_categorical
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from keras.preprocessing.image import ImageDataGenerator
from skimage import measure
from skimage.transform import rotate
import tensorflow as tf
from imutils import contours
import imutils
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [0]:
df_train = pd.read_csv(path+"training.csv")
df_test = pd.read_csv(path+"test.csv")

In [0]:
cols = ['x1','x2','y1','y2']
cords = df_train[cols]
cords = np.asarray(cords)
#cords_scaled = cords/5

In [0]:
#imgs = imgs/255
cord_new_180 = np.array([0,0,0,0])
cord_new_mirror = np.array([0,0,0,0])
#print(cord_new.shape)
for i in range(14000):
    img = imgs[i]
    new_img = imutils.rotate(img,180)
    imgs.append(new_img)
    cord_new_180[0] = 640 - cords[i][1]
    cord_new_180[1] = 640 - cords[i][0]
    cord_new_180[2] = 480 - cords[i][3]
    cord_new_180[3] = 480 - cords[i][2]
    cords = np.vstack([cords, cord_new_180])

  #print(cord_new)

for i in range(14000):
    img = imgs[i]
    fimg = cv2.flip(img,1)
    imgs.append(fimg)
    cord_new_mirror[0] = 640 - cords[i][1]
    cord_new_mirror[1] = 640 - cords[i][0]
    cord_new_mirror[2] = cords[i][2]
    cord_new_mirror[3] = cords[i][3]
    cords = np.vstack([cords, cord_new_mirror])

In [0]:
cords_scaled = cords/5
imgs = np.asarray(imgs)
print(imgs.shape)

In [0]:
X_train,X_val,y_train,y_val = train_test_split(imgs,cords_scaled,test_size = 0.1,random_state=42)

X_train = np.asarray(X_train)
X_val = np.asarray(X_val)
y_train = np.asarray(y_train)
y_val = np.asarray(y_val)
print(X_train.shape,X_val.shape,y_train.shape,y_val.shape)

In [0]:
class LossHistory(Callback):
    def on_train_begin(self,logs={}):
        self.losses=[]
        self.lr = []
        
    def on_epoch_end(self,batch,logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(exp_decay(len(self.losses)))

def exp_decay(epoch):
    initial_lrate = 0.01
    k = 0.1
    lrate = initial_lrate * exp(-k*epoch)
    return lrate

lrate = LearningRateScheduler(exp_decay)

class PlotLearning(Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.IoU = []
        self.val_IoU = []
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.IoU.append(logs.get('IoU'))
        self.val_IoU.append(logs.get('val_IoU'))
        self.i += 1
        f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
        
        clear_output(wait=True)
        
        ax1.set_yscale('log')
        ax1.plot(self.x, self.losses, label="loss")
        ax1.plot(self.x, self.val_losses, label="val_loss")
        ax1.legend()
        
        ax2.plot(self.x, self.IoU, label="IoU")
        ax2.plot(self.x, self.val_IoU, label="validation IoU")
        ax2.legend()
        
        plt.show();
        
plot = PlotLearning()

loss_history = LossHistory()
#callbacks_list = [loss_history,lrate,plot]
callbacks_list = [loss_history,plot,ModelCheckpoint(path+'extra/custom_model_iou_2.{epoch:02d}-{val_loss:.2f}.h5',monitor = 'val_loss',verbose = 1,save_best_only=False,save_weights_only = False, mode = 'auto',period=5)]

In [0]:
def calculate_iou(y_true, y_pred):
    results = []
    
    for i in range(0,y_true.shape[0]):
    
        y_true = y_true.astype(np.float32)
        y_pred = y_pred.astype(np.float32)

        x1 = y_true[0,0] 
        x2 = y_true[0,1]
        y1 = y_true[0,2]
        y2 = y_true[0,3]
        area_boxTrue = ((x2-x1) * (y2-y1))

        x1p = y_pred[0,0]
        x2p = y_pred[0,1]
        y1p = y_pred[0,2]
        y2p = y_pred[0,3]
        area_boxPred = ((x2p-x1p) * (y2p-y1p))

        x_boxTrue_br = x2
        y_boxTrue_br = y2 

        x_boxPred_br = x2p
        y_boxPred_br = y2p 

        x_boxInt_tleft = np.max([x1,x1p])
        y_boxInt_tleft = np.max([y1,y1p])

        x_boxInt_br = np.min([x2,x2p])
        y_boxInt_br = np.min([y2,y2p]) 
        
        area_of_intersection = np.max([0,(x_boxInt_br - x_boxInt_tleft)]) * np.max([0,(y_boxInt_br - y_boxInt_tleft)])

        iou = area_of_intersection / ((area_boxTrue + area_boxPred) - area_of_intersection)

        iou = iou.astype(np.float32)
        
        results.append(iou)
    
    return np.mean(results)
  
def IoU(y_true, y_pred):
    
    iou = tf.py_func(calculate_iou, [y_true, y_pred], tf.float32)

    return iou

In [0]:
def locmodel():
    model = Sequential()
  
    model.add(Conv2D(32, (3, 3), padding = 'same',strides=(1,1),kernel_initializer = 'he_uniform',input_shape=(96, 128, 3)))
    model.add(BatchNormalization(axis = 3))
    model.add(Activation('relu'))

    #model.add(Conv2D(32, (3, 3),padding = 'same',strides=(1,1),kernel_initializer = 'he_uniform'))
    #model.add(BatchNormalization(axis = 3))
    #model.add(Activation('relu'))
  
    model.add(Conv2D(32, (3, 3), padding = 'same',strides=(1,1),kernel_initializer = 'he_uniform'))
    model.add(BatchNormalization(axis = 3))
    model.add(Activation('relu'))

    model.add(MaxPool2D(pool_size=(2, 2)))
    #model.add(Conv2D(96,(3,3),strides = 2))
  
    #model.add(Conv2D(64, (3, 3), padding = 'same',strides=(1,1),kernel_initializer = 'he_uniform'))
    #model.add(BatchNormalization(axis = 3))
    #model.add(Activation('relu'))

    model.add(Conv2D(64, (3, 3), padding = 'same',strides=(1,1),kernel_initializer = 'he_uniform'))
    model.add(BatchNormalization(axis = 3))
    model.add(Activation('relu'))
  
    model.add(Conv2D(64, (3, 3), padding = 'same',strides=(1,1),kernel_initializer = 'he_uniform'))
    model.add(BatchNormalization(axis = 3))
    model.add(Activation('relu'))

    model.add(MaxPool2D(pool_size=(2, 2)))
    #model.add(Conv2D(192,(3,3),strides = 2))
  
    #model.add(Conv2D(128, (3, 3), padding = 'same',strides=(1,1),kernel_initializer = 'he_uniform'))
    #model.add(BatchNormalization(axis = 3))
    #model.add(Activation('relu'))
  
    model.add(Conv2D(128, (3, 3), padding = 'same',strides=(1,1),kernel_initializer = 'he_uniform'))
    model.add(BatchNormalization(axis = 3))
    model.add(Activation('relu'))
  
    model.add(Conv2D(128, (3, 3), padding = 'same',strides=(1,1),kernel_initializer = 'he_uniform'))
    model.add(BatchNormalization(axis = 3))
    model.add(Activation('relu'))

    model.add(MaxPool2D(pool_size=(2, 2)))
  
    #model.add(Conv2D(256, (3, 3), padding = 'same',strides=(1,1),kernel_initializer = 'he_uniform'))
    #model.add(BatchNormalization(axis = 3))
    #model.add(Activation('relu'))

    model.add(Conv2D(256, (3, 3), padding = 'same',strides=(1,1),kernel_initializer = 'he_uniform'))
    model.add(BatchNormalization(axis = 3))
    model.add(Activation('relu'))
  
    model.add(Conv2D(256, (3, 3), padding = 'same',strides=(1,1),kernel_initializer = 'he_uniform'))
    model.add(BatchNormalization(axis = 3))
    model.add(Activation('relu'))

    model.add(MaxPool2D(pool_size=(2, 2)))
  
    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dropout(0.1))
  
    model.add(Dense(4))
    model.add(Activation('relu')) 

    model.compile(optimizer='adam', loss='mean_squared_error',metrics = [IoU])
    print(model.summary())
    
    return model

In [0]:
locmodel = locmodel()

In [0]:
locmodel.fit(X_train,y_train,validation_data = (X_val,y_val),epochs=2,callbacks=callbacks_list,batch_size=8)
#locmodel.save_weights(path+'custom_model_iou_2.hdf5')
#locmodel.save(path+"custom_model_iou_2.h5")

In [0]:
with open('/content/drive/My Drive/Flipkart/output_test.bin', 'rb') as f:
    imgs_test = pickle.load(f)
    
imgs_test = np.asarray(imgs_test)
imgs_test.shape

In [0]:
predictions = locmodel.predict(imgs_test)

In [0]:
predictions_original = predictions*5

In [0]:
for i in range(len(predictions_original)):
    predictions_original[i][0] = int(round(predictions_original[i][0]))
    predictions_original[i][1] = int(round(predictions_original[i][1]))
    predictions_original[i][2] = int(round(predictions_original[i][2]))
    predictions_original[i][3] = int(round(predictions_original[i][3]))

In [0]:
def make_rectangle(img,cords):
    #print(cords)
    x1 = int(round(cords[0]))
    x2 = int(round(cords[1]))
    y1 = int(round(cords[2]))
    y2 = int(round(cords[3]))
    #print(x1,y1,x2,y2)
  
    if x2==128:
        x2 -= 1
    if y2==96:
        y2 -= 1
    
    img_new = img
    
    for i in range(x1,x2+1):
        img_new[y1][i] = np.asarray([255,0,0])
        img_new[y2][i] = np.asarray([255,0,0])
    
    for i in range(y1,y2+1):
        img_new[i][x1] = np.asarray([255,0,0])
        img_new[i][x2] = np.asarray([255,0,0])    
    
    return img_new

In [0]:
i=1800
img_new0 = make_rectangle(imgs_test[i],predictions[i])
plt.imshow(img_new0)

In [0]:
df_test['x1'] = predictions_original[:,0]
df_test['x2'] = predictions_original[:,1]
df_test['y1'] = predictions_original[:,2]
df_test['y2'] = predictions_original[:,3]
df_test.head()

In [0]:
for i in range(len(df_test)):
    if df_test['x2'][i]>640:
        print(i)
        df_test['x2'][i] = 640.0
    
for i in range(len(df_test)):
    if df_test['y2'][i]>480:
        print(i)
        df_test['y2'][i] = 480.0

In [0]:
df_test.to_csv(path+"submission.csv",index=False)